### image dataset load

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

data_dir = r'/content/drive/My Drive/기업프로젝트-라젠/data/40, 320'
print(len(os.listdir(data_dir)))

IMAGE_SIZE = (224,224)
BATCH_SIZE = 128

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
    )

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
    )

320
Found 13120 files belonging to 320 classes.
Using 10496 files for training.
Found 13120 files belonging to 320 classes.
Using 2624 files for validation.


In [ ]:
print(train_ds)
print(val_ds)
print(len(train_ds))
print(len(val_ds))

<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>
<BatchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>
82
21


### 데이터 정규화

In [ ]:
from tensorflow.keras import layers

# 0~1 사이의 값을 가지도록 조정
def normalization_dataset(dataset):
    normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
    normalized_ds = dataset.map(lambda x, y: (normalization_layer(x), y))
    return normalized_ds 

train_ds = normalization_dataset(train_ds)
val_ds = normalization_dataset(val_ds)
print(train_ds)
print(val_ds)

<MapDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>
<MapDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>


### 성능을위한 데이터 세트 구성

In [ ]:
# cache() 첫 번째 에포크 동안 디스크에서로드 된 이미지를 메모리에 보관합니다. 
# prefetch 학습 중에 데이터 전처리 및 모델 실행과 겹칩니다

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


### tensorflow module load

In [ ]:
import tensorflow_hub as hub
import tensorflow_addons as tfa
 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Lambda, MaxPooling2D, Conv2D
from tensorflow.keras.optimizers import Adam, SGD, Adagrad

from tensorflow.keras.callbacks import EarlyStopping

### 모델 load, train

In [ ]:
# mobilenet_v2
hub_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/classification/4"
hub_layer = hub.KerasLayer(hub_url,
                           input_shape=IMAGE_SIZE + (3,),
                           trainable=True, 
                           arguments=dict(batch_norm_momentum=0.997))

# model구축
# triplets 128벡터값을 위한 dense출력값을 128로 설정, activation = None
# deep architecture이후 l2정규화
model = Sequential()
model.add(hub_layer)
model.add(Dense(128, activation=None))
model.add(Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))  

model.summary()

# 모델 complie
adam = Adam(lr=0.001, decay=1e-6)
model.compile(optimizer=adam, loss=tfa.losses.TripletSemiHardLoss())

# SGD = SGD(lr=0.05)
# model.compile(optimizer=SGD, loss=tfa.losses.TripletSemiHardLoss())

# Adagrad = Adagrad(lr=0.01, epsilon=1e-6)
# model.compile(optimizer=adam, loss=tfa.losses.TripletSemiHardLoss())

# earlystopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)


# 모델 학습
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=1000,
                    callbacks=[early_stopping],
                    )  

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1001)              1692489   
_________________________________________________________________
dense (Dense)                (None, 128)               128256    
_________________________________________________________________
lambda (Lambda)              (None, 128)               0         
Total params: 1,820,745
Trainable params: 1,806,665
Non-trainable params: 14,080
_________________________________________________________________
Epoch 1/1000
82/82 [==============================] - ETA: 0s - loss: 1.0820  

KeyboardInterrupt: ignored

In [ ]:
##### Earl Stopping 삭제한 코드


# mobilenet_v2
hub_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_035_224/classification/4"
hub_layer = hub.KerasLayer(hub_url,
                           input_shape=IMAGE_SIZE + (3,),
                           trainable=True, 
                           arguments=dict(batch_norm_momentum=0.997))

# model구축
# triplets 128벡터값을 위한 dense출력값을 128로 설정, activation = None
# deep architecture이후 l2정규화
model = Sequential()
model.add(hub_layer)
model.add(Dense(128, activation=None))
model.add(Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))  

model.summary()

# 모델 complie
adam = Adam(lr=0.001, decay=1e-6)
model.compile(optimizer=adam, loss=tfa.losses.TripletSemiHardLoss())

# SGD = SGD(lr=0.05)
# model.compile(optimizer=SGD, loss=tfa.losses.TripletSemiHardLoss())

# Adagrad = Adagrad(lr=0.01, epsilon=1e-6)
# model.compile(optimizer=adam, loss=tfa.losses.TripletSemiHardLoss())

# earlystopping
# early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)


# 모델 학습
history = model.fit(train_ds,
                    validation_data = val_ds,
                    epochs=1000,
                    # callbacks=[early_stopping],
                    )  

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1001)              1692489   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               128256    
_________________________________________________________________
lambda_1 (Lambda)            (None, 128)               0         
Total params: 1,820,745
Trainable params: 1,806,665
Non-trainable params: 14,080
_________________________________________________________________
Epoch 1/1000
 2/41 [>.............................] - ETA: 27s - loss: 1.0937WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1314s vs `on_train_batch_end` time: 0.6491s). Check your callbacks.


41/41 [==============================] - 33s 793ms/step - loss: 1.0923 - val_loss: 1.0985
Epoch 2/1000
41/41 [==============================] - 33s 795ms/step - loss: 1.0858 - val_loss: 1.0963
Epoch 3/1000
41/41 [==============================] - 32s 788ms/step - loss: 1.0720 - val_loss: 1.0942
Epoch 4/1000
41/41 [==============================] - 33s 796ms/step - loss: 1.0601 - val_loss: 1.0950
Epoch 5/1000
41/41 [==============================] - 32s 781ms/step - loss: 1.0458 - val_loss: 1.0907
Epoch 6/1000
41/41 [==============================] - 31s 767ms/step - loss: 1.0278 - val_loss: 1.0880
Epoch 7/1000
41/41 [==============================] - 32s 775ms/step - loss: 1.0104 - val_loss: 1.0881
Epoch 8/1000
41/41 [==============================] - 32s 782ms/step - loss: 0.9933 - val_loss: 1.0885
Epoch 9/1000
41/41 [==============================] - 33s 796ms/step - loss: 0.9724 - val_loss: 1.0880
Epoch 10/1000
41/41 [==============================] - 32s 786ms/step - loss: 0.9548 -

In [ ]:
model.save('/content/drive/My Drive/기업프로젝트-라젠/model/mobilenet_v2-2.h5')

NameError: ignored